In [1]:
exec(open("./090120_autoencoderhelperfunctions.py").read())
total_df = pd.read_pickle('/users/madke/documents/060519qabegen.pkl')

C:\Users\madke\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
n3 = pd.read_csv('./092820_stikhivecs.csv', index_col = 0)
n4 = n3[~n3.text.str.contains('і')].copy()
n5 = n4[~n4.text.str.contains('стихия')].copy()
n6 = n5[~n5.text.str.contains('стихии')].copy()
n7 = n6[~n6.text.str.contains('стихает')].copy()

In [3]:
n7 = n7.dropna()
n7 = n7.reset_index(drop = True)
R1 = n7.drop(['entry_id','text'], axis = 1)
R1 = R1.dropna()
R1 = R1.fillna(0)
x_train = R1.values

In [4]:
np.random.seed(1)
tf.compat.v1.random.set_random_seed(1)

activ_h = 'relu'
KI_h = 'he_uniform' 

L1 = 75
L2 = 45
L3 = 15
L4 = 3

v_dim = 100
activ_v = 'tanh'
learn = 0.0004
opt = tf.optimizers.Adam(lr= learn, amsgrad = True)
loss_fctn = 'cosine_similarity'
en_name = str(L1) + '_' + str(L2) + '_' + str(L3) 

model = Sequential()
#encoder
model.add(Dense(L1, input_dim=v_dim, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L2, input_dim=L1, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L3, input_dim=L2, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L4, input_dim=L3, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L4, activation=activ_v, name = 'encoded'))

#decoder
model.add(Dense(L3, input_dim=L4, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L2, input_dim=L3, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L1, input_dim=L2, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(v_dim, input_dim=L1, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(v_dim, activation=activ_v, name = 'ypred'))


model.compile(loss=loss_fctn, optimizer=opt, metrics=['acc', 'mae', 'msle', 'mse'])

ran for 10000 epochs found peak in val loss and accuracy around 250 epochs (0.923, 0.50)  

In [ ]:
history = model.fit(x_train, x_train,
          batch_size=10,
          epochs=5000, verbose = 1, validation_split = 0.2)

xprednp = layer_extract('ypred', model, x_train)
xtrainnp = np.array(x_train)
accs = get_cos(xtrainnp, xprednp)
print('Median exp:' + str(np.median(accs)))

np.random.shuffle(xprednp)
np.random.shuffle(xtrainnp)
accsR = get_cos(xprednp, xtrainnp)
print('Median random:' + str(np.median(accsR)))

Train on 16476 samples, validate on 4119 samples
Epoch 1/5000
16476/16476 [==============================] - 3s 200us/step - loss: -0.9045 - acc: 0.4172 - mae: 0.0677 - msle: 0.0024 - mse: 0.0072 - val_loss: -0.9121 - val_acc: 0.4688 - val_mae: 0.0626 - val_msle: 0.0021 - val_mse: 0.0062
Epoch 2/5000
16476/16476 [==============================] - 2s 147us/step - loss: -0.9096 - acc: 0.4190 - mae: 0.0600 - msle: 0.0018 - mse: 0.0056 - val_loss: -0.9125 - val_acc: 0.4688 - val_mae: 0.0574 - val_msle: 0.0017 - val_mse: 0.0052
Epoch 3/5000
16476/16476 [==============================] - 2s 140us/step - loss: -0.9097 - acc: 0.4208 - mae: 0.0557 - msle: 0.0016 - mse: 0.0048 - val_loss: -0.9125 - val_acc: 0.4688 - val_mae: 0.0538 - val_msle: 0.0015 - val_mse: 0.0045
Epoch 4/5000
16476/16476 [==============================] - 3s 173us/step - loss: -0.9098 - acc: 0.4209 - mae: 0.0527 - msle: 0.0014 - mse: 0.0043 - val_loss: -0.9127 - val_acc: 0.4688 - val_mae: 0.0512 - val_msle: 0.0014 - val_mse

Epoch 68/5000
16476/16476 [==============================] - 2s 141us/step - loss: -0.9324 - acc: 0.4911 - mae: 0.0434 - msle: 0.0013 - mse: 0.0049 - val_loss: -0.9327 - val_acc: 0.5011 - val_mae: 0.0387 - val_msle: 9.3895e-04 - val_mse: 0.0034
Epoch 69/5000
16476/16476 [==============================] - 2s 136us/step - loss: -0.9325 - acc: 0.4913 - mae: 0.0444 - msle: 0.0013 - mse: 0.0052 - val_loss: -0.9336 - val_acc: 0.5062 - val_mae: 0.0412 - val_msle: 0.0011 - val_mse: 0.0042
Epoch 70/5000
16476/16476 [==============================] - 2s 135us/step - loss: -0.9327 - acc: 0.4905 - mae: 0.0448 - msle: 0.0013 - mse: 0.0052 - val_loss: -0.9332 - val_acc: 0.5018 - val_mae: 0.0408 - val_msle: 0.0011 - val_mse: 0.0039
Epoch 71/5000
16476/16476 [==============================] - 2s 141us/step - loss: -0.9329 - acc: 0.4953 - mae: 0.0487 - msle: 0.0015 - mse: 0.0060 - val_loss: -0.9335 - val_acc: 0.5006 - val_mae: 0.0428 - val_msle: 0.0011 - val_mse: 0.0042
Epoch 72/5000
16476/16476 [=====

Epoch 102/5000
16476/16476 [==============================] - 2s 137us/step - loss: -0.9356 - acc: 0.4945 - mae: 0.0664 - msle: 0.0024 - mse: 0.0097 - val_loss: -0.9353 - val_acc: 0.5091 - val_mae: 0.0635 - val_msle: 0.0022 - val_mse: 0.0086
Epoch 103/5000
16476/16476 [==============================] - 2s 142us/step - loss: -0.9356 - acc: 0.4941 - mae: 0.0699 - msle: 0.0027 - mse: 0.0108 - val_loss: -0.9357 - val_acc: 0.5081 - val_mae: 0.0665 - val_msle: 0.0024 - val_mse: 0.0093
Epoch 104/5000
16476/16476 [==============================] - 2s 138us/step - loss: -0.9356 - acc: 0.4918 - mae: 0.0690 - msle: 0.0026 - mse: 0.0106 - val_loss: -0.9354 - val_acc: 0.5101 - val_mae: 0.0620 - val_msle: 0.0021 - val_mse: 0.0082
Epoch 105/5000
16476/16476 [==============================] - 2s 148us/step - loss: -0.9356 - acc: 0.4927 - mae: 0.0688 - msle: 0.0026 - mse: 0.0105 - val_loss: -0.9357 - val_acc: 0.5108 - val_mae: 0.0645 - val_msle: 0.0023 - val_mse: 0.0088
Epoch 106/5000
16476/16476 [====

Epoch 136/5000
16476/16476 [==============================] - 3s 157us/step - loss: -0.9366 - acc: 0.4954 - mae: 0.0782 - msle: 0.0033 - mse: 0.0135 - val_loss: -0.9364 - val_acc: 0.5047 - val_mae: 0.0766 - val_msle: 0.0031 - val_mse: 0.0127
Epoch 137/5000
16476/16476 [==============================] - 3s 157us/step - loss: -0.9367 - acc: 0.4933 - mae: 0.0785 - msle: 0.0033 - mse: 0.0136 - val_loss: -0.9362 - val_acc: 0.5050 - val_mae: 0.0763 - val_msle: 0.0030 - val_mse: 0.0121
Epoch 138/5000
16476/16476 [==============================] - 2s 144us/step - loss: -0.9367 - acc: 0.4920 - mae: 0.0752 - msle: 0.0030 - mse: 0.0124 - val_loss: -0.9359 - val_acc: 0.5057 - val_mae: 0.0710 - val_msle: 0.0027 - val_mse: 0.0107
Epoch 139/5000
16476/16476 [==============================] - 3s 160us/step - loss: -0.9366 - acc: 0.4947 - mae: 0.0744 - msle: 0.0030 - mse: 0.0124 - val_loss: -0.9364 - val_acc: 0.5101 - val_mae: 0.0702 - val_msle: 0.0026 - val_mse: 0.0105
Epoch 140/5000
16476/16476 [====

Epoch 170/5000
16476/16476 [==============================] - 2s 139us/step - loss: -0.9373 - acc: 0.4961 - mae: 0.0889 - msle: 0.0040 - mse: 0.0168 - val_loss: -0.9371 - val_acc: 0.5113 - val_mae: 0.0889 - val_msle: 0.0040 - val_mse: 0.0164
Epoch 171/5000
16476/16476 [==============================] - 2s 138us/step - loss: -0.9374 - acc: 0.4981 - mae: 0.0889 - msle: 0.0040 - mse: 0.0168 - val_loss: -0.9369 - val_acc: 0.5125 - val_mae: 0.0780 - val_msle: 0.0032 - val_mse: 0.0128
Epoch 172/5000
16476/16476 [==============================] - 2s 135us/step - loss: -0.9375 - acc: 0.5001 - mae: 0.0853 - msle: 0.0038 - mse: 0.0158 - val_loss: -0.9367 - val_acc: 0.5123 - val_mae: 0.0804 - val_msle: 0.0034 - val_mse: 0.0142
Epoch 173/5000
16476/16476 [==============================] - 2s 134us/step - loss: -0.9373 - acc: 0.4993 - mae: 0.0897 - msle: 0.0041 - mse: 0.0173 - val_loss: -0.9365 - val_acc: 0.5084 - val_mae: 0.0762 - val_msle: 0.0031 - val_mse: 0.0127
Epoch 174/5000
16476/16476 [====

...

Epoch 1834/5000
16476/16476 [==============================] - 2s 149us/step - loss: -0.9471 - acc: 0.5279 - mae: 0.1348 - msle: 0.0072 - mse: 0.0303 - val_loss: -0.9430 - val_acc: 0.5181 - val_mae: 0.1357 - val_msle: 0.0073 - val_mse: 0.0306
Epoch 1835/5000
16476/16476 [==============================] - 2s 140us/step - loss: -0.9470 - acc: 0.5342 - mae: 0.1341 - msle: 0.0071 - mse: 0.0300 - val_loss: -0.9429 - val_acc: 0.5181 - val_mae: 0.1330 - val_msle: 0.0071 - val_mse: 0.0297
Epoch 1836/5000
16476/16476 [==============================] - 2s 146us/step - loss: -0.9471 - acc: 0.5339 - mae: 0.1353 - msle: 0.0073 - mse: 0.0306 - val_loss: -0.9428 - val_acc: 0.5249 - val_mae: 0.1338 - val_msle: 0.0072 - val_mse: 0.0301
Epoch 1837/5000
16476/16476 [==============================] - 2s 141us/step - loss: -0.9470 - acc: 0.5339 - mae: 0.1348 - msle: 0.0072 - mse: 0.0304 - val_loss: -0.9427 - val_acc: 0.5244 - val_mae: 0.1323 - val_msle: 0.0070 - val_mse: 0.0292
Epoch 1838/5000
16476/16476 

Epoch 1868/5000
16476/16476 [==============================] - 2s 149us/step - loss: -0.9472 - acc: 0.5291 - mae: 0.1361 - msle: 0.0073 - mse: 0.0308 - val_loss: -0.9428 - val_acc: 0.5256 - val_mae: 0.1387 - val_msle: 0.0076 - val_mse: 0.0319
Epoch 1869/5000
16476/16476 [==============================] - 2s 150us/step - loss: -0.9474 - acc: 0.5332 - mae: 0.1366 - msle: 0.0074 - mse: 0.0310 - val_loss: -0.9431 - val_acc: 0.5198 - val_mae: 0.1353 - val_msle: 0.0073 - val_mse: 0.0304
Epoch 1870/5000
16476/16476 [==============================] - 2s 146us/step - loss: -0.9473 - acc: 0.5313 - mae: 0.1351 - msle: 0.0072 - mse: 0.0304 - val_loss: -0.9427 - val_acc: 0.5229 - val_mae: 0.1323 - val_msle: 0.0070 - val_mse: 0.0293
Epoch 1871/5000
16476/16476 [==============================] - 2s 149us/step - loss: -0.9473 - acc: 0.5332 - mae: 0.1343 - msle: 0.0072 - mse: 0.0300 - val_loss: -0.9431 - val_acc: 0.5198 - val_mae: 0.1334 - val_msle: 0.0071 - val_mse: 0.0297
Epoch 1872/5000
16476/16476 

Epoch 1902/5000
16476/16476 [==============================] - 2s 148us/step - loss: -0.9473 - acc: 0.5327 - mae: 0.1391 - msle: 0.0076 - mse: 0.0322 - val_loss: -0.9428 - val_acc: 0.5242 - val_mae: 0.1363 - val_msle: 0.0074 - val_mse: 0.0310
Epoch 1903/5000
16476/16476 [==============================] - 2s 143us/step - loss: -0.9474 - acc: 0.5346 - mae: 0.1392 - msle: 0.0076 - mse: 0.0322 - val_loss: -0.9427 - val_acc: 0.5205 - val_mae: 0.1352 - val_msle: 0.0073 - val_mse: 0.0305
Epoch 1904/5000
16476/16476 [==============================] - 2s 148us/step - loss: -0.9474 - acc: 0.5307 - mae: 0.1390 - msle: 0.0076 - mse: 0.0321 - val_loss: -0.9430 - val_acc: 0.5195 - val_mae: 0.1384 - val_msle: 0.0076 - val_mse: 0.0320
Epoch 1905/5000
16476/16476 [==============================] - 3s 167us/step - loss: -0.9471 - acc: 0.5331 - mae: 0.1422 - msle: 0.0079 - mse: 0.0336 - val_loss: -0.9429 - val_acc: 0.5188 - val_mae: 0.1409 - val_msle: 0.0078 - val_mse: 0.0329
Epoch 1906/5000
16476/16476 

Epoch 1936/5000
16476/16476 [==============================] - 2s 144us/step - loss: -0.9471 - acc: 0.5301 - mae: 0.1342 - msle: 0.0072 - mse: 0.0301 - val_loss: -0.9430 - val_acc: 0.5242 - val_mae: 0.1321 - val_msle: 0.0070 - val_mse: 0.0293
Epoch 1937/5000
16476/16476 [==============================] - 2s 147us/step - loss: -0.9471 - acc: 0.5286 - mae: 0.1346 - msle: 0.0072 - mse: 0.0302 - val_loss: -0.9430 - val_acc: 0.5212 - val_mae: 0.1346 - val_msle: 0.0072 - val_mse: 0.0303
Epoch 1938/5000
16476/16476 [==============================] - 2s 148us/step - loss: -0.9472 - acc: 0.5278 - mae: 0.1350 - msle: 0.0072 - mse: 0.0304 - val_loss: -0.9429 - val_acc: 0.5186 - val_mae: 0.1349 - val_msle: 0.0073 - val_mse: 0.0306
Epoch 1939/5000
16476/16476 [==============================] - 2s 144us/step - loss: -0.9471 - acc: 0.5308 - mae: 0.1360 - msle: 0.0073 - mse: 0.0309 - val_loss: -0.9427 - val_acc: 0.5249 - val_mae: 0.1338 - val_msle: 0.0071 - val_mse: 0.0300
Epoch 1940/5000
16476/16476 

Epoch 1970/5000
16476/16476 [==============================] - 4s 226us/step - loss: -0.9477 - acc: 0.5308 - mae: 0.1358 - msle: 0.0073 - mse: 0.0308 - val_loss: -0.9432 - val_acc: 0.5271 - val_mae: 0.1350 - val_msle: 0.0073 - val_mse: 0.0304
Epoch 1971/5000
16476/16476 [==============================] - 4s 215us/step - loss: -0.9476 - acc: 0.5309 - mae: 0.1350 - msle: 0.0072 - mse: 0.0304 - val_loss: -0.9431 - val_acc: 0.5195 - val_mae: 0.1361 - val_msle: 0.0074 - val_mse: 0.0308
Epoch 1972/5000
16476/16476 [==============================] - 2s 149us/step - loss: -0.9474 - acc: 0.5315 - mae: 0.1346 - msle: 0.0072 - mse: 0.0302 - val_loss: -0.9430 - val_acc: 0.5232 - val_mae: 0.1333 - val_msle: 0.0071 - val_mse: 0.0296
Epoch 1973/5000
16476/16476 [==============================] - 3s 169us/step - loss: -0.9472 - acc: 0.5296 - mae: 0.1337 - msle: 0.0071 - mse: 0.0297 - val_loss: -0.9430 - val_acc: 0.5222 - val_mae: 0.1322 - val_msle: 0.0070 - val_mse: 0.0291
Epoch 1974/5000
16476/16476 

In [13]:
og = layer_extract('encoded', model, x_train)
make_graph(np.array(og), n7['text'], n7['entry_id'], name = en_name + activ_h + '.html', PCA = False)

In [14]:
sub_total = total_df[total_df['entry_id'].isin(list(n7['entry_id']))][['diarynum', 'entry_year', 'entry_id']]
sub_total = sub_total.reset_index(drop = True)
sub_total2 = pd.concat([n7['entry_id'], pd.DataFrame(og)], axis = 1)
sub_total = sub_total.merge(sub_total2, left_on = 'entry_id', right_on = 'entry_id', how = 'inner')
sub_total.to_csv('./100520_stikhi.csv')